# Laboratorio: Métodos de búsqueda

# Mariano Villanueva Aguilar 24/11/2024

En las clases anteriores creaste códigos para realizar búsquedas aleatorias (Simulated Annealing) y búsquedas dirigidas (Optimización Bayesiana). Estos métodos de búsqueda se utilizan para facilitar el proceso de optimización de funciones objetivos compleja y costosas de computar.

En este laboratorio usaremos el dataset de los diferentes tipos de iris, y sus longitudes y anchos de pétalos y sépalos. Utilizaremos un RandomForest para crear un modelo de clasificación y el métrico F1 para decidir cuál es el mejor modelo de acuerdo a lo que tenemos disponible.

1. Carga el dataset de Iris

In [6]:
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

2. Importa el archivo `Bosque.py`.

Este archivo contiene la función `RegresionBosque`, que recibe:
- X: las características independientes
- y: la variable de respuesta
- árboles: cantidad total de árboles
- profundidad de bosque: niveles de profundidad del bosque

Su salida es:
- modelo: El objeto con el modelo ajustado
- f1: El métrico que califica qué tan bueno es el modelo que se ajustó.


In [8]:
import Bosque
modelo, f1 = Bosque.RegresionBosque(X, y, 10, 3)
f1

0.9444444444444444

### Actividad 1:

Inicializa un espacio con 5 muestras en nuestro dominio de variables independientes:
- árboles: números enteros entre 5 y 50.
- profundidad: números enteros entre 2 y 10

Utiliza optimización Bayesiana para encontrar la combinación de árboles y profundidad que **maximice** el métrico F1.

In [26]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
ARB = np.random.randint(5,50,5)
PROF= np.random.randint(2,10,5)
ARB,PROF

(array([14,  6, 18,  7, 34]), array([7, 4, 3, 5, 5]))

In [28]:
f1_lista=[]
for i in range (len(PROF)):
    modelo, f1 = Bosque.RegresionBosque(X, y, ARB[i], PROF[i])
    f1_lista.append(f1)
f1_lista


[0.9333333333333333,
 0.9444444444444444,
 0.9333333333333333,
 0.9555555555555556,
 0.9444444444444444]

In [30]:
arb_vec = ARB.reshape([-1, 1])
prof_vec = PROF.reshape([-1, 1])
X_hiperparam = np.hstack((arb_vec, prof_vec))
X_hiperparam

array([[14,  7],
       [ 6,  4],
       [18,  3],
       [ 7,  5],
       [34,  5]])

In [32]:
kernel=1.0*RBF(length_scale=1)
gp= GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(X_hiperparam,f1_lista)

In [34]:
x1=np.linspace(0,50,100).reshape([-1,1])
x2=np.linspace(2,10,100).reshape([-1,1])
Matrizmatricial=np.hstack((x1, x2))
y_pred,y_std=gp.predict(Matrizmatricial,return_std=True)
y_pred_high=y_pred+1.96*y_std
i_next=np.argmax(y_pred_high)
new_X=np.vstack((X_hiperparam,Matrizmatricial[i_next]))
new_X

array([[14.,  7.],
       [ 6.,  4.],
       [18.,  3.],
       [ 7.,  5.],
       [34.,  5.],
       [50., 10.]])

In [36]:
Matrizmatricial[i_next]

array([50., 10.])

In [38]:
Modelo,f1=Bosque.RegresionBosque(X, y, 50, 10)


In [40]:
f1_lista.append(f1)
f1_lista

[0.9333333333333333,
 0.9444444444444444,
 0.9333333333333333,
 0.9555555555555556,
 0.9444444444444444,
 0.9555555555555556]

In [42]:
#Segunda iteración
kernel=1.0*RBF(length_scale=1)
gp= GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(new_X,f1_lista)
y_pred,y_std=gp.predict(Matrizmatricial,return_std=True)
y_pred_high=y_pred+1.96*y_std
i_next=np.argmax(y_pred_high)
new_X2=np.vstack((new_X,Matrizmatricial[i_next]))
new_X2

array([[14.        ,  7.        ],
       [ 6.        ,  4.        ],
       [18.        ,  3.        ],
       [ 7.        ,  5.        ],
       [34.        ,  5.        ],
       [50.        , 10.        ],
       [45.45454545,  9.27272727]])

In [44]:
Matrizmatricial[i_next]

array([45.45454545,  9.27272727])

In [46]:
Modelo,f1=Bosque.RegresionBosque(X, y,45 ,9)
f1_lista.append(f1)
f1_lista

[0.9333333333333333,
 0.9444444444444444,
 0.9333333333333333,
 0.9555555555555556,
 0.9444444444444444,
 0.9555555555555556,
 0.9555555555555556]

In [48]:
#Tercera iteración
kernel=1.0*RBF(length_scale=1)
gp= GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(new_X2,f1_lista)
y_pred,y_std=gp.predict(Matrizmatricial,return_std=True)
y_pred_high=y_pred+1.96*y_std
i_next=np.argmax(y_pred_high)
new_X3=np.vstack((new_X2,Matrizmatricial[i_next]))
new_X3

array([[14.        ,  7.        ],
       [ 6.        ,  4.        ],
       [18.        ,  3.        ],
       [ 7.        ,  5.        ],
       [34.        ,  5.        ],
       [50.        , 10.        ],
       [45.45454545,  9.27272727],
       [47.97979798,  9.67676768]])

In [50]:
Matrizmatricial[i_next]

array([47.97979798,  9.67676768])

In [52]:
Modelo,f1=Bosque.RegresionBosque(X, y,48 ,10)
f1_lista.append(f1)
f1_lista

[0.9333333333333333,
 0.9444444444444444,
 0.9333333333333333,
 0.9555555555555556,
 0.9444444444444444,
 0.9555555555555556,
 0.9555555555555556,
 0.9555555555555556]

In [54]:
#Cuarta iteración
kernel=1.0*RBF(length_scale=1)
gp= GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(new_X3,f1_lista)
y_pred,y_std=gp.predict(Matrizmatricial,return_std=True)
y_pred_high=y_pred+1.96*y_std
i_next=np.argmax(y_pred_high)
new_X4=np.vstack((new_X3,Matrizmatricial[i_next]))
new_X4

array([[14.        ,  7.        ],
       [ 6.        ,  4.        ],
       [18.        ,  3.        ],
       [ 7.        ,  5.        ],
       [34.        ,  5.        ],
       [50.        , 10.        ],
       [45.45454545,  9.27272727],
       [47.97979798,  9.67676768],
       [32.32323232,  7.17171717]])

In [56]:
Matrizmatricial[i_next]

array([32.32323232,  7.17171717])

In [58]:
Modelo,f1=Bosque.RegresionBosque(X, y,32 ,7)
f1_lista.append(f1)
f1_lista

[0.9333333333333333,
 0.9444444444444444,
 0.9333333333333333,
 0.9555555555555556,
 0.9444444444444444,
 0.9555555555555556,
 0.9555555555555556,
 0.9555555555555556,
 0.9444444444444444]

In [60]:
#Quinta iteración
kernel=1.0*RBF(length_scale=1)
gp= GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(new_X4,f1_lista)
y_pred,y_std=gp.predict(Matrizmatricial,return_std=True)
y_pred_high=y_pred+1.96*y_std
i_next=np.argmax(y_pred_high)
new_X5=np.vstack((new_X4,Matrizmatricial[i_next]))
new_X5

array([[14.        ,  7.        ],
       [ 6.        ,  4.        ],
       [18.        ,  3.        ],
       [ 7.        ,  5.        ],
       [34.        ,  5.        ],
       [50.        , 10.        ],
       [45.45454545,  9.27272727],
       [47.97979798,  9.67676768],
       [32.32323232,  7.17171717],
       [46.46464646,  9.43434343]])

In [62]:
Matrizmatricial[i_next]

array([46.46464646,  9.43434343])

In [68]:
Modelo,f1=Bosque.RegresionBosque(X, y,46 ,9)
f1_lista.append(f1)
f1_lista

[0.9333333333333333,
 0.9444444444444444,
 0.9333333333333333,
 0.9555555555555556,
 0.9444444444444444,
 0.9555555555555556,
 0.9555555555555556,
 0.9555555555555556,
 0.9444444444444444,
 0.9555555555555556]

In [70]:
#Sexta iteración
kernel=1.0*RBF(length_scale=1)
gp= GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(new_X5,f1_lista)
y_pred,y_std=gp.predict(Matrizmatricial,return_std=True)
y_pred_high=y_pred+1.96*y_std
i_next=np.argmax(y_pred_high)
new_X6=np.vstack((new_X5,Matrizmatricial[i_next]))
new_X6

array([[14.        ,  7.        ],
       [ 6.        ,  4.        ],
       [18.        ,  3.        ],
       [ 7.        ,  5.        ],
       [34.        ,  5.        ],
       [50.        , 10.        ],
       [45.45454545,  9.27272727],
       [47.97979798,  9.67676768],
       [32.32323232,  7.17171717],
       [46.46464646,  9.43434343],
       [46.46464646,  9.43434343]])

In [72]:
Matrizmatricial[i_next]

array([46.46464646,  9.43434343])

In [74]:
Modelo,f1=Bosque.RegresionBosque(X, y,47 ,10)
f1_lista.append(f1)
f1_lista

[0.9333333333333333,
 0.9444444444444444,
 0.9333333333333333,
 0.9555555555555556,
 0.9444444444444444,
 0.9555555555555556,
 0.9555555555555556,
 0.9555555555555556,
 0.9444444444444444,
 0.9555555555555556,
 0.9555555555555556]

### Actividad 2:

Inicializa 2 vectores con posibles valores para las variables independientes:
- árboles: números enteros entre 5 y 50
- profundidad: números enteros entre 2 y 10

Utiliza el algoritmo de Simulated Annealing que siga el siguiente orden:
- Elige un punto de partida para las variables.
- Selecciona al azar una de las dos para modificarlas.
- Selecciona un elemento al azar de la lista que contiene los posibles valores de esa variable.
- Sigue el algoritmo ($p$ y $q$) para decidir si usas esa combinación nueva o si mantienes la anterior.

In [113]:
arb=np.random.randint(5,51)
prof=np.random.randint(2,11)
juntos=np.array([arb,prof])
juntos

array([48,  8])

In [159]:
N=30
T=1000
M=100
Larboles=[]
Lprof=[]
Lrutas=[]
for i in range (M):
    R1=np.array([np.random.randint(5,51),np.random.randint(2,11)])
    while(T>=1):
        for j in range (N):
            decision=np.random.randint(0,2)
            R2=R1.copy()
            if decision==0:
                R2[decision]=np.random.randint(5,51)
            else:
                R2[decision]=np.random.randint(2,11)
                
            p=np.random.uniform(0,1)
            D1=R1[decision]
            D2=R2[decision]
            q=np.exp((D1-D2)/T)
            if p<q:
                R1=R2.copy()     
        T=T*0.9
    Larboles.append(R1[0].copy())
    Lprof.append(R1[1].copy())
    Lrutas.append(R1.copy())
Lrutas


[array([5, 3]),
 array([49,  7]),
 array([42,  3]),
 array([46,  8]),
 array([15,  6]),
 array([27,  2]),
 array([38,  8]),
 array([38,  2]),
 array([34,  5]),
 array([35,  4]),
 array([12,  5]),
 array([35,  7]),
 array([9, 6]),
 array([13,  8]),
 array([30, 10]),
 array([21,  7]),
 array([16,  3]),
 array([43,  2]),
 array([25,  7]),
 array([14,  6]),
 array([23,  3]),
 array([11,  5]),
 array([37,  8]),
 array([7, 4]),
 array([43,  8]),
 array([50,  8]),
 array([6, 4]),
 array([16,  2]),
 array([30,  4]),
 array([47,  2]),
 array([18,  8]),
 array([31,  5]),
 array([37,  6]),
 array([35,  4]),
 array([24,  9]),
 array([42,  6]),
 array([14,  9]),
 array([46,  9]),
 array([25,  9]),
 array([7, 5]),
 array([28,  7]),
 array([50,  3]),
 array([12,  8]),
 array([6, 4]),
 array([6, 4]),
 array([36,  4]),
 array([17,  2]),
 array([48,  9]),
 array([26,  8]),
 array([45,  9]),
 array([10, 10]),
 array([46,  3]),
 array([21,  8]),
 array([7, 9]),
 array([33,  9]),
 array([46,  6]),
 array([

[0, 1]